**Import libraries**

In [6]:
#https://grouplens.org/datasets/movielens/

**Initiate spark session**

In [7]:
!pip install pyspark
import pandas as pd
from pyspark.sql.functions import col, explode
from pyspark import SparkContext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 55 kB/s 
     |████████████████████████████████| 199 kB 77.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=9e1759fb5bc8424ba7b99cc9200285f1462822375120c40ddff9699606951acc
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [8]:
from pyspark.sql import SparkSession
sc = SparkContext
# sc.setCheckpointDir('checkpoint')
spark = SparkSession.builder.appName('Recommendations').getOrCreate()

**Load data**

In [9]:
movies = spark.read.csv("/content/sample_data/movies.csv",header=True)
ratings = spark.read.csv("/content/sample_data/ratings.csv",header=True)

In [10]:
ratings.show()

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|     4|964982703|
|     1|      3|     4|964981247|
|     1|      6|     4|964982224|
|     1|     47|     5|964983815|
|     1|     50|     5|964982931|
|     1|     70|     3|964982400|
|     1|    101|     5|964980868|
|     1|    110|     4|964982176|
|     1|    151|     5|964984041|
|     1|    157|     5|964984100|
|     1|    163|     5|964983650|
|     1|    216|     5|964981208|
|     1|    223|     3|964980985|
|     1|    231|     5|964981179|
|     1|    235|     4|964980908|
|     1|    260|     5|964981680|
|     1|    296|     3|964982967|
|     1|    316|     3|964982310|
|     1|    333|     5|964981179|
|     1|    349|     4|964982563|
+------+-------+------+---------+
only showing top 20 rows



In [11]:
ratings.printSchema()

root
 |-- userId: string (nullable = true)
 |-- movieId: string (nullable = true)
 |-- rating: string (nullable = true)
 |-- timestamp: string (nullable = true)



In [12]:
ratings = ratings.\
    withColumn('userId', col('userId').cast('integer')).\
    withColumn('movieId', col('movieId').cast('integer')).\
    withColumn('rating', col('rating').cast('float')).\
    drop('timestamp')
ratings.show()

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
|     1|    163|   5.0|
|     1|    216|   5.0|
|     1|    223|   3.0|
|     1|    231|   5.0|
|     1|    235|   4.0|
|     1|    260|   5.0|
|     1|    296|   3.0|
|     1|    316|   3.0|
|     1|    333|   5.0|
|     1|    349|   4.0|
+------+-------+------+
only showing top 20 rows



**Calculate sparsity**

In [13]:
# Count the total number of ratings in the dataset
numerator = ratings.select("rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = ratings.select("userId").distinct().count()
num_movies = ratings.select("movieId").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.30% empty.


**Interpret ratings**

In [14]:
# Group data by userId, count ratings
userId_ratings = ratings.groupBy("userId").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
|userId|count|
+------+-----+
|   414| 2698|
|   599| 2478|
|   474| 2108|
|   448| 1864|
|   274| 1346|
|   610| 1302|
|    68| 1260|
|   380| 1218|
|   606| 1115|
|   288| 1055|
|   249| 1046|
|   387| 1027|
|   182|  977|
|   307|  975|
|   603|  943|
|   298|  939|
|   177|  904|
|   318|  879|
|   232|  862|
|   480|  836|
+------+-----+
only showing top 20 rows



In [15]:
# Group data by userId, count ratings
movieId_ratings = ratings.groupBy("movieId").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
|movieId|count|
+-------+-----+
|    356|  329|
|    318|  317|
|    296|  307|
|    593|  279|
|   2571|  278|
|    260|  251|
|    480|  238|
|    110|  237|
|    589|  224|
|    527|  220|
|   2959|  218|
|      1|  215|
|   1196|  211|
|   2858|  204|
|     50|  204|
|     47|  203|
|    780|  202|
|    150|  201|
|   1198|  200|
|   4993|  198|
+-------+-----+
only showing top 20 rows



**Build Out An ALS Model**

In [16]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [17]:
# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)

pyspark.ml.recommendation.ALS

**Tell Spark how to tune your ALS model**

In [18]:
# Import the requisite items
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \

           
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  16


**Build your cross validation pipeline**

In [19]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

CrossValidator_1f763d724161


**Best Model and Best Model Parameters**

In [20]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train);

#Extract best model from the cv model above
best_model = model.bestModel

In [21]:
# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 50
  MaxIter: 10
  RegParam: 0.15


In [22]:
# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

0.868142452308257


In [23]:
test_predictions.show()

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   148|    356|   4.0| 3.5239072|
|   148|   4896|   4.0|  3.510039|
|   148|   4993|   3.0| 3.4699442|
|   148|   7153|   3.0| 3.4208226|
|   148|   8368|   4.0| 3.5653324|
|   148|  40629|   5.0|  3.301226|
|   148|  50872|   3.0| 3.6407778|
|   148|  60069|   4.5| 3.7034664|
|   148|  69757|   3.5| 3.4164522|
|   148|  72998|   4.0| 3.2564936|
|   148|  81847|   4.5|  3.451922|
|   148|  98491|   5.0| 3.7482162|
|   148| 115617|   3.5| 3.6247354|
|   148| 122886|   3.5|  3.372223|
|   463|    296|   4.0| 4.1569595|
|   463|    527|   4.0| 3.7740083|
|   463|   2019|   4.0| 3.9213715|
|   471|    527|   4.5| 3.8166275|
|   471|   6016|   4.0|  3.990181|
|   471|   6333|   2.5| 3.2488956|
+------+-------+------+----------+
only showing top 20 rows



In [24]:
# Generate n Recommendations for all users
nrecommendations = best_model.recommendForAllUsers(10)
nrecommendations.limit(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|     1|[{3379, 5.761998}...|
|     3|[{70946, 4.857783...|
|     5|[{3379, 4.61099},...|
|     6|[{42730, 4.764389...|
|     9|[{5490, 4.755606}...|
|    12|[{42730, 5.606037...|
|    13|[{3379, 4.9655128...|
|    15|[{3379, 4.4231443...|
|    16|[{3379, 4.5773177...|
|    17|[{3379, 5.1072655...|
+------+--------------------+



In [25]:
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

nrecommendations.limit(10).show()

+------+-------+---------+
|userId|movieId|   rating|
+------+-------+---------+
|     1|   3379| 5.761998|
|     1|  33649|5.5904136|
|     1|   5490|5.4927096|
|     1| 171495|  5.42172|
|     1|   5416| 5.376472|
|     1|   5328| 5.376472|
|     1|   3951| 5.376472|
|     1|   6460| 5.332468|
|     1|   5915|5.3250713|
|     1|  78836|5.3197055|
+------+-------+---------+



**Do the recommendations make sense? **

Lets merge movie name and genres to teh recommendation matrix for interpretability.

In [26]:
nrecommendations.join(movies, on='movieId').filter('userId = 100').show()

+-------+------+---------+--------------------+--------------------+
|movieId|userId|   rating|               title|              genres|
+-------+------+---------+--------------------+--------------------+
|  67618|   100|  5.06765|Strictly Sexual (...|Comedy|Drama|Romance|
|  33649|   100|5.0600514|  Saving Face (2004)|Comedy|Drama|Romance|
|   3379|   100| 4.961345| On the Beach (1959)|               Drama|
|  42730|   100| 4.961083|   Glory Road (2006)|               Drama|
|  26073|   100|4.9105597|Human Condition I...|           Drama|War|
|   7071|   100|4.9105597|Woman Under the I...|               Drama|
|  84273|   100|4.9105597|Zeitgeist: Moving...|         Documentary|
| 179135|   100|4.9105597|Blue Planet II (2...|         Documentary|
| 184245|   100|4.9105597|De platte jungle ...|         Documentary|
| 117531|   100|4.9105597|    Watermark (2014)|         Documentary|
+-------+------+---------+--------------------+--------------------+



In [27]:
ratings.join(movies, on='movieId').filter('userId = 100').sort('rating', ascending=False).limit(10).show()

+-------+------+------+--------------------+--------------------+
|movieId|userId|rating|               title|              genres|
+-------+------+------+--------------------+--------------------+
|   1101|   100|   5.0|      Top Gun (1986)|      Action|Romance|
|   1958|   100|   5.0|Terms of Endearme...|        Comedy|Drama|
|   2423|   100|   5.0|Christmas Vacatio...|              Comedy|
|   4041|   100|   5.0|Officer and a Gen...|       Drama|Romance|
|   5620|   100|   5.0|Sweet Home Alabam...|      Comedy|Romance|
|    368|   100|   4.5|     Maverick (1994)|Adventure|Comedy|...|
|    934|   100|   4.5|Father of the Bri...|              Comedy|
|    539|   100|   4.5|Sleepless in Seat...|Comedy|Drama|Romance|
|     16|   100|   4.5|       Casino (1995)|         Crime|Drama|
|    553|   100|   4.5|    Tombstone (1993)|Action|Drama|Western|
+-------+------+------+--------------------+--------------------+

